# Introdução à Ciência de Dados - Trabalho Módulo 1

## Criando um dataset de carros usados coletados a partir da OLX

In [1]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
import os
from tqdm import tqdm


In [2]:
BASE_URL = 'https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios'
HEADERS = {
        'authority': 'olx.com.br',
        'method': 'GET',
        'scheme': 'https',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
}

In [3]:
def collect_links(n_pages=5):    
    total_pages = get_number_pages()
    
    if n_pages == 'all' or n_pages > total_pages:
        n_pages = total_pages

    for page in tqdm(range(n_pages)):
        url = f'{BASE_URL}?o={page+1}'
        response = requests.get(url=url, headers=HEADERS)
        
        soup = BeautifulSoup(response.content, 'lxml')
        items = soup.find_all('li', {'class': 'sc-1fcmfeb-2 juiJqh'})
        
        for item in items:
            try:
                with open('links.txt', 'a') as outfile:
                    outfile.write(item.findAll('a')[0]['href']+'\n')
            except IndexError:
                continue

In [4]:
def get_number_pages():
    response = requests.get(url=BASE_URL, headers=HEADERS)
    soup = BeautifulSoup(response.content, 'lxml')
    results = soup.findAll('span', {'class': 'sc-1mi5vq6-0 eDXljX sc-ifAKCX fhJlIo'})

    ITEMS_PER_PAGE = 50
    n_cars_found = float(results[0].contents[0].split('de ')[-1].split(' ')[0].replace('.', ''))

    return int(n_cars_found/ITEMS_PER_PAGE)

In [5]:
def proccess_links(file):
    cars = []
    with open('links.txt', 'r') as infile:
        links = infile.readlines()
        
        for link in tqdm(links):
            response = requests.get(url=link, headers=HEADERS)
            soup = BeautifulSoup(response.content, 'lxml')
            
            items = soup.find_all('div', {'class': 'sc-hmzhuo eNZSNe sc-jTzLTM iwtnNi'})
            
            car = {}
            # Caracteristics
            for item in items:
                title = item.find_all('span', {'class': 'sc-ifAKCX dCObfG'})[0].contents[0]
                try:
                    value = item.find_all('span', {'class': 'sc-ifAKCX cmFKIN'})[0].contents[0]
                except:
                    value = item.find_all('a', {'class': 'sc-57pm5w-0 XtcoW'})[0].contents[0]
                
                car[title] = value
        
            locations = soup.find_all('div', {'class': 'sc-hmzhuo sc-1f2ug0x-3 ONRJp sc-jTzLTM iwtnNi'})
            
            # Location
            for location in locations:
                title = location.find_all('dt', {'class': 'sc-1f2ug0x-0 cLGFbW sc-ifAKCX cmFKIN'})[0].contents[0]
                value = location.find_all('dd', {'class': 'sc-1f2ug0x-1 ljYeKO sc-ifAKCX kaNiaQ'})[0].contents[0]

                car[title] = value
            
            car['url'] = link.strip()
            cars.append(car)

    df = pd.DataFrame(cars)
    df.to_csv('olx_cars.csv')

In [ ]:
links_file = 'links.txt'
if not os.path.exists(links_file):
    collect_links(n_pages='all')
else:
    proccess_links(links_file)

  0%|          | 21/17293 [00:26<7:01:11,  1.46s/it]